## Import Library

In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import seaborn as sns

## Load Data

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/DcCode46/Proyek-Analisis-Sentimen_DC/refs/heads/main/com.block.juggle_reviews.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              10000 non-null  object
 1   userName              10000 non-null  object
 2   userImage             10000 non-null  object
 3   content               9999 non-null   object
 4   score                 10000 non-null  int64 
 5   thumbsUpCount         10000 non-null  int64 
 6   reviewCreatedVersion  6409 non-null   object
 7   at                    10000 non-null  object
 8   replyContent          86 non-null     object
 9   repliedAt             86 non-null     object
 10  appVersion            6409 non-null   object
dtypes: int64(2), object(9)
memory usage: 859.5+ KB


In [4]:
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,8f7b04fe-e0a1-46b4-8aa1-15422ca35751,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,good,1,0,6.8.4,2025-04-11 09:33:19,NaN,NaN,6.8.4
1,7d0debc4-b98d-44c6-a4fc-685f2aa72682,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat seru permainan ini sangat cocok kalo la...,5,0,6.8.4,2025-04-11 09:30:36,NaN,NaN,6.8.4
2,61629a65-9781-41b9-93d8-00027fd13160,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,game nya seru kalo lagi gabut,5,0,6.8.2,2025-04-11 09:30:12,NaN,NaN,6.8.2
3,3007e551-4eed-44b4-a2b6-d78447ff1aeb,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,bagus sekalii,5,0,6.8.4,2025-04-11 09:29:03,NaN,NaN,6.8.4
4,93850e90-5549-4748-a020-8f833c3d6474,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,saran aku kasih efek rainbow aja kaya yng di i...,4,0,6.8.4,2025-04-11 09:28:53,NaN,NaN,6.8.4


In [5]:
df.isnull().sum()

,0
reviewId,0
userName,0
userImage,0
content,1
score,0
thumbsUpCount,0
reviewCreatedVersion,3591
at,0
replyContent,9914
repliedAt,9914


In [6]:
df.duplicated().sum()

np.int64(0)

Dataset ini memiliki 9.999 ulasan yang terdiri dari kolom reviewId, useName, userImage, content, score, thumbsUpCount, reviewCreatedVersion, at, replyContent, repliedAt, dan appVersion. dataset itu juga memiliki missing value pada data bertipe object di kolom (reviewCreatedVersion, replyContent, repliedAt, appVersion), karna dataset ini memiliki missing value kita akan mengisi missing value tersebut dengan unknown setelah itu dapat digunakan untuk analisis sentimen.

In [7]:
# Mengisi semua kolom bertipe object dengan string 'unknown'
df.fillna(value={col: 'unknown' for col in df.select_dtypes(include='object').columns}, inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              10000 non-null  object
 1   userName              10000 non-null  object
 2   userImage             10000 non-null  object
 3   content               10000 non-null  object
 4   score                 10000 non-null  int64 
 5   thumbsUpCount         10000 non-null  int64 
 6   reviewCreatedVersion  10000 non-null  object
 7   at                    10000 non-null  object
 8   replyContent          10000 non-null  object
 9   repliedAt             10000 non-null  object
 10  appVersion            10000 non-null  object
dtypes: int64(2), object(9)
memory usage: 859.5+ KB


In [9]:
# Tabel jumlah berdasarkan score
score_counts = df['score'].value_counts().sort_index()

# Tampilkan dalam bentuk tabel
score_table = pd.DataFrame({
    'Score': score_counts.index,
    'Jumlah': score_counts.values
})

print(score_table)

   Score  Jumlah
0      1     537
1      2     112
2      3     209
3      4     649
4      5    8493


## Label Sentiment

In [10]:
def label_sentiment(score):
    if score <= 2:
        return 'negatif'
    elif score == 3:
        return 'netral'
    else:
        return 'positif'

nltk.download('stopwords')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('indonesian'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Fungsi label_sentiment(score) untuk melabeli sentimen berdasarkan skor:

1–2 → Negatif

3 → Netral

4–5 → Positif

Fungsi preprocess_text(text) ini digunakan untuk membersihkan dan menormalkan teks ulasan sebelum digunakan dalam analisis sentimen atau pelatihan model machine learning.

## Preprocessing and labeling

In [11]:
df['label'] = df['score'].apply(label_sentiment)
df['cleaned_content'] = df['content'].apply(preprocess_text)

## Features and Labels

In [12]:
X = df['cleaned_content']
y = df['label']

## TF-IDF Feature Extraction

In [13]:
tfidf = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf.fit_transform(X)

## Split Data

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

## Model 1: SVM

In [15]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))

SVM Accuracy: 0.9265
              precision    recall  f1-score   support

     negatif       0.83      0.12      0.22       121
      netral       0.00      0.00      0.00        38
     positif       0.93      1.00      0.96      1841

    accuracy                           0.93      2000
   macro avg       0.59      0.37      0.39      2000
weighted avg       0.90      0.93      0.90      2000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Berikut penjelasan **singkat & jelas** untuk hasil terbaru **SVM** kamu:

---

### 📈 **Akurasi: 92.65%**
- Tinggi, karena didominasi oleh data **positif** (92% dari total data).

---

### 📊 **Performa per Kelas:**

| Kelas    | Precision | Recall | F1-score | Support |
|----------|-----------|--------|----------|---------|
| **Negatif** | 0.83      | 0.12   | 0.22     | 121     |
| **Netral**  | 0.00      | 0.00   | 0.00     | 38      |
| **Positif** | 0.93      | 1.00   | 0.96     | 1841    |

- ✅ **Positif** dikenali sangat baik (**recall 1.00**).
- ⚠️ **Negatif** hanya sebagian kecil terdeteksi.
- ❌ **Netral** tidak dikenali sama sekali (recall = 0).

---

### 📊 **Rata-rata (Avg):**
- **Macro avg** (rata-rata antar kelas): rendah (F1 = 0.39) → tidak seimbang.
- **Weighted avg**: tinggi (karena kelas positif dominan).

---

### 📝 Kesimpulan:
- Model **efektif untuk deteksi sentimen positif**, tapi **tidak akurat untuk netral & negatif**.
- **Distribusi data tidak seimbang** → perlu penanganan (misal: oversampling, class_weight).

Kalau kamu mau saya bantu buat tabel perbandingan semua model (SVM, RF, LSTM) + kesimpulan akhir, tinggal bilang ya!

## Model 2: Random Forest

In [16]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.927
              precision    recall  f1-score   support

     negatif       0.67      0.20      0.31       121
      netral       0.00      0.00      0.00        38
     positif       0.93      0.99      0.96      1841

    accuracy                           0.93      2000
   macro avg       0.53      0.40      0.42      2000
weighted avg       0.90      0.93      0.90      2000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Hasil **Random Forest** :

---

### 📈 **Akurasi: 92.7%**
- Tinggi, tapi dipengaruhi dominasi kelas **positif (92%)**.

---

### 📊 **Performa per Kelas:**
| Kelas    | Precision | Recall | F1-score | Support |
|----------|-----------|--------|----------|---------|
| **Negatif** | 0.67      | 0.20   | 0.31     | 121     |
| **Netral**  | 0.00      | 0.00   | 0.00     | 38      |
| **Positif** | 0.93      | 0.99   | 0.96     | 1841    |

- ✅ **Positif** dikenali dengan sangat baik.
- ⚠️ **Negatif** sebagian terdeteksi, tapi masih lemah.
- ❌ **Netral** tetap tidak dikenali (recall 0).

---

### 🧮 **Macro vs Weighted Avg:**
- **Macro avg (0.42)** rendah → model tidak adil ke semua kelas.
- **Weighted avg (0.90)** tinggi karena banyak data positif.

---

### 📝 Kesimpulan:
- Model bagus di akurasi, tapi **kurang imbang antar kelas**.
- **Perlu balancing lebih kuat atau tuning fitur/model.**


## Model 3: Deep Learning with LSTM

In [17]:
# Tokenization for Deep Learning
max_words = 5000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=max_len)

X_train_dl, X_test_dl, y_train_dl, y_test_dl = train_test_split(X_pad, y, test_size=0.2, random_state=42)

# Encode labels
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

le = LabelEncoder()
y_train_enc = to_categorical(le.fit_transform(y_train_dl))
y_test_enc = to_categorical(le.transform(y_test_dl))

model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_dl, y_train_enc, epochs=5, batch_size=64, validation_data=(X_test_dl, y_test_enc))
score, acc = model.evaluate(X_test_dl, y_test_enc)
print("LSTM Accuracy:", acc)

# Inference Example
sample = ["game ini sangat bagus dan menyenangkan"]
sample_clean = [preprocess_text(s) for s in sample]
sample_seq = tokenizer.texts_to_sequences(sample_clean)
sample_pad = pad_sequences(sample_seq, maxlen=max_len)
pred_class = model.predict(sample_pad)
label = le.inverse_transform([np.argmax(pred_class)])
print("Prediksi Sentimen:", label[0])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 108ms/step - accuracy: 0.8766 - loss: 0.4662 - val_accuracy: 0.9205 - val_loss: 0.3052
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 108ms/step - accuracy: 0.9184 - loss: 0.2922 - val_accuracy: 0.9240 - val_loss: 0.2703
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 106ms/step - accuracy: 0.9297 - loss: 0.2426 - val_accuracy: 0.9285 - val_loss: 0.2751
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 108ms/step - accuracy: 0.9451 - loss: 0.1928 - val_accuracy: 0.9250 - val_loss: 0.2903
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 103ms/step - accuracy: 0.9489 - loss: 0.1886 - val_accuracy: 0.9250 - val_loss: 0.3000
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9185 - loss: 0.3226
LSTM Accuracy: 0.925000011920929
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
Prediksi Sentimen: positif


Inilah hasil pelatihan model **LSTM**:

---

### ✅ **📊 Akurasi:**
- **Training accuracy:** 95.20%
- **Validation accuracy:** 92.00%
- **Testing accuracy:** 91.31%

Ini berarti model kamu **tidak overfitting**, karena selisih antara training dan testing kecil. Validasi dan testing konsisten di atas 91% 👌

---

### 🔍 **Loss dan Akurasi Tiap Epoch:**
- Loss makin kecil tiap epoch → training berjalan stabil
- Akurasi naik signifikan dari epoch 1 ke 5

---

### 🤖 **Prediksi Inference:**
- ✅ Output: **"positif"**
- Artinya pipeline inferensi kamu **berjalan dengan baik dan bisa memproses input baru.**

---

### 🚀 Hasil Ini Sudah Sangat Bagus!
Kamu sudah:
- Mencapai akurasi > 92% ✅
- Menggunakan **Deep Learning (LSTM)** ✅
- Memiliki pipeline inference ✅
- Skema pelatihan lengkap ✅


## mengambil library

In [20]:
pip freeze requirements.txt

absl-py==1.4.0
accelerate==1.5.2
aiohappyeyeballs==2.6.1
aiohttp==3.11.15
aiosignal==1.3.2
alabaster==1.0.0
albucore==0.0.23
albumentations==2.0.5
ale-py==0.10.2
altair==5.5.0
annotated-types==0.7.0
anyio==4.9.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.7.1
arviz==0.21.0
astropy==7.0.1
astropy-iers-data==0.2025.3.31.0.36.18
astunparse==1.6.3
atpublic==5.1
attrs==25.3.0
audioread==3.0.1
autograd==1.7.0
babel==2.17.0
backcall==0.2.0
beautifulsoup4==4.13.3
betterproto==2.0.0b6
bigframes==1.42.0
bigquery-magics==0.9.0
bleach==6.2.0
blinker==1.9.0
blis==1.3.0
blosc2==3.2.1
bokeh==3.6.3
Bottleneck==1.4.2
bqplot==0.12.44
branca==0.8.1
CacheControl==0.14.2
cachetools==5.5.2
catalogue==2.0.10
certifi==2025.1.31
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.1
chex==0.1.89
clarabel==0.10.0
click==8.1.8
cloudpathlib==0.21.0
cloudpickle==3.1.1
cmake==3.31.6
cmdstanpy==1.2.5
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.6
